<a href="https://colab.research.google.com/github/KrishnaSChavan/MachineLearning/blob/main/Clav2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [ ]:
from google. colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed

100% 6.40G/6.40G [05:40<00:00, 16.2MB/s]
100% 6.40G/6.40G [05:40<00:00, 20.2MB/s]


In [5]:
import zipfile
zip_r = zipfile.ZipFile('/content/cervical-cancer-largest-dataset-sipakmed.zip')
zip_r.extractall()
zip_r.close()

In [6]:
!mkdir Data

In [7]:
one = "/content/im_Dyskeratotic/im_Dyskeratotic/"
two = "/content/im_Koilocytotic/im_Koilocytotic/"
three = "/content/im_Metaplastic/im_Metaplastic/"
four = "/content/im_Parabasal/im_Parabasal/CROPPED/"
five = "/content/im_Superficial-Intermediate/im_Superficial-Intermediate/"

In [8]:
path = [one,two,three,four,five]

In [9]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
data = []
labels = []
height = 64
width = 64
channels = 3
classes = 43
n_inputs = height * width * channels

i = 0
for p in path:
    Class=os.listdir(p)
    for a in Class:
        if(a[-1] == 'p'):
            labels.append(i)
            try:
                print(p+a)
                image=cv2.imread(p+a)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((height, width))
                data.append(np.array(size_image))
            except:
                print('')
    i+=1

In [13]:
len(data),len(labels)

(1645, 1645)

In [14]:
labels = np.array(labels)
data = np.array(data)

In [15]:
data.shape

(1645, 64, 64, 3)

In [20]:
labels

array([0, 0, 0, ..., 4, 4, 4])

In [19]:
categories = ["Dyskeratotic", "Koilocytotic", "Metaplastic", "Parabasal", "Superficial-Intermediate"]

In [17]:
def img(i):
    plt.imshow(data[i])
    plt.xlabel(categories[labels[i]])

In [30]:
np.random.seed(42)
s = np.arange(data.shape[0])
np.random.shuffle(s)
data = data[s]
labels = labels[s]

In [49]:
from keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam

In [32]:
resnet = ResNet50(weights='imagenet',include_top = False)

94765736/94765736 [==============================] - 5s 0us/step


In [47]:
IMAGE_SHAPE = (64,64)
def create_model(model_url, num_classes=5):

    feature_extraction_layer = hub.KerasLayer(model_url, trainable=False, name="feature_extraction", input_shape=IMAGE_SHAPE+(3,))

    # create our own model
    model = Sequential()
    model.add(feature_extraction_layer)
    model.add(Dense(num_classes, activation='softmax', name='output'))

    return model

In [41]:
scaled = data/255.

In [44]:
scaled.shape

(1645, 64, 64, 3)

In [45]:
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

In [48]:
resnet_model = create_model(resnet_url)

In [69]:
# resnet_model.compile(
#     loss = "categorical_crossentropy",
#     optimizer = Adam(),
#     metrics=['accuracy']
# )
resnet_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Make sure this loss is appropriate for your task
    metrics=['accuracy'],
    run_eagerly=True  # Set to True to execute the model eagerly
)


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled, labels, test_size = 0.2, random_state = 1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1316, 64, 64, 3)
(329, 64, 64, 3)
(1316,)
(329,)


In [51]:
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
    log_dir = dir_name+'/'+experiment_name+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"saving tensorboard file to {log_dir}")
    return tensorboard_callback

In [86]:
from keras.utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=5)
y_test_encoded = to_categorical(y_test, num_classes=5)
y = to_categorical(labels, num_classes=5)

In [90]:
# resnet_history=resnet_model.fit(
#             X_train,
#             y_test,
#             epochs = 10,
#             steps_per_epoch =len(X_train),
#             validation_data=X_test,
#             validation_steps=len(X_test),
#             callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',experiment_name='resnet50 v2')]
# )
resnet_history = resnet_model.fit(
    tdt,  # Assuming y_train contains the corresponding labels for X_train
    epochs=10,
)


Epoch 1/10
52/52 [==============================] - 9s 121ms/step - loss: 1.0072 - accuracy: 0.6717
Epoch 2/10
52/52 [==============================] - 6s 115ms/step - loss: 0.4406 - accuracy: 0.8298
Epoch 3/10
52/52 [==============================] - 5s 102ms/step - loss: 0.3451 - accuracy: 0.8717
Epoch 4/10
52/52 [==============================] - 6s 114ms/step - loss: 0.3010 - accuracy: 0.8942
Epoch 5/10
52/52 [==============================] - 6s 106ms/step - loss: 0.2632 - accuracy: 0.9058
Epoch 6/10
52/52 [==============================] - 6s 115ms/step - loss: 0.2429 - accuracy: 0.9185
Epoch 7/10
52/52 [==============================] - 5s 103ms/step - loss: 0.2235 - accuracy: 0.9246
Epoch 8/10
52/52 [==============================] - 5s 104ms/step - loss: 0.2114 - accuracy: 0.9343
Epoch 9/10
52/52 [==============================] - 6s 112ms/step - loss: 0.1997 - accuracy: 0.9368
Epoch 10/10
52/52 [==============================] - 8s 149ms/step - loss: 0.1884 - accuracy: 0.9398

In [78]:
x_flattened = np.reshape(data, (data.shape[0], -1))

In [79]:


X_train, X_test, Y_train, Y_test = train_test_split(x_flattened , labels, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (-1, data.shape[1], data.shape[2], data.shape[3]))
X_test = np.reshape(X_test, (-1, data.shape[1], data.shape[2], data.shape[3]))

In [56]:
X_train.shape

(1316, 64, 64, 3)

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [87]:
from sklearn.utils import shuffle
IMAGE_SHAPE = (64,64)
BATCH_SIZE = 32
data_gen = ImageDataGenerator(rescale=(1/255.))

tdt = data_gen.flow(
    x = data,
    y=y,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [88]:
len(tdt)

52

In [72]:
X_train.shape

(1316, 64, 64, 3)

In [73]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(x_flattened)
principal_X = pca.transform(x_flattened)

In [100]:
zzz = cv2.imread("/content/im_Dyskeratotic/im_Dyskeratotic/001.bmp")
image_from_array = Image.fromarray(zzz, 'RGB')
aaa = image_from_array.resize((height, width))

In [101]:
aaa = np.array(aaa)

In [106]:
aaa.shape

(64, 64, 3)

In [107]:
aaa = aaa/255.

In [109]:
reshaped_aaa = np.reshape(aaa, (-1, 64, 64, 3))


In [112]:
asd = resnet_model.predict(reshaped_aaa)

1/1 [==============================] - 0s 26ms/step


In [97]:
zzz = np.array(zzz)

In [98]:
zzz.shape

()

In [113]:
np.argmax(asd, axis=1)

array([0])

In [114]:
resnet_model.save("CLAV")

In [115]:
resnet_model.save("CLAV2",save_format="tf")

In [116]:
resnet_model.save('CLAV3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [125]:
loaded_model = tf.keras.models.load_model("CLAV3.h5")

ValueError: ignored

In [120]:
new_model = tf.keras.models.load_model('CLAV')

In [136]:
import zipfile

# File or directory to be zipped
file_or_directory_to_zip = '/content/CLAV2'

# Name for the zip file
zip_filename = 'CLAVZ2.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    if os.path.isfile(file_or_directory_to_zip):
        # If it's a single file, add it to the archive
        zipf.write(file_or_directory_to_zip, os.path.basename(file_or_directory_to_zip))
    elif os.path.isdir(file_or_directory_to_zip):
        # If it's a directory, add all files within the directory and its subdirectories
        for root, dirs, files in os.walk(file_or_directory_to_zip):
            for file in files:
                file_to_zip = os.path.join(root, file)
                archive_name = os.path.relpath(file_to_zip, file_or_directory_to_zip)
                zipf.write(file_to_zip, archive_name)

# The zip archive is created with the specified name


In [127]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()

In [128]:
!mkdir CLAVZ

In [129]:
!cd CLAVZ

In [130]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()

In [135]:
new_model_1 = tf.keras.models.load_model('/content/CLAVZ')

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_r = zipfile.ZipFile('/content/CLAVZ.zip')
zip_r.extractall()
zip_r.close()